In [1]:
from keras import applications
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras import regularizers
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.utils.np_utils import to_categorical 
from keras.callbacks import ModelCheckpoint
from keras import Model
from keras import initializers
from keras.callbacks import LearningRateScheduler
from keras.utils import layer_utils, np_utils
from keras.applications.xception import preprocess_input
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.datasets import make_classification
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.utils import class_weight
import seaborn as sn
from scipy import interp
from itertools import cycle
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from scipy.sparse import csr_matrix

Using TensorFlow backend.
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
image_width = 224
image_height = 224

# Change the batchsize according to your system RAM
batch_size = 64

train_dir = "C:/Users/hp/Desktop/Diabetic_retinopathy_dataset_kaggle/original_new/"

vgg16_base = applications.VGG16(weights='imagenet', include_top=False, input_shape=(image_width, image_height, 3))

In [3]:
x = vgg16_base.get_layer(index=-1).output
feature_extraction_layer = GlobalAveragePooling2D()(x)
model = Model(inputs=vgg16_base.input, outputs=feature_extraction_layer)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [4]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_width, image_height),
        batch_size=batch_size,
        class_mode=None,  # this means our generator will only yield batches of data, no labels
        shuffle=False)

nb_train_samples = len(train_generator.filenames)  
num_classes = len(train_generator.class_indices)

# get the class lebels for the training data, in the original order  
train_labels = train_generator.classes  
   
# convert the training labels to categorical vectors  
train_labels = to_categorical(train_labels, num_classes=num_classes)

Found 744 images belonging to 5 classes.


In [5]:
# # the predict_generator method returns the output of a model, given
# # a generator that yields batches of numpy data
# bottleneck_features_train = model.predict_generator(train_generator, nb_train_samples // batch_size + 1)
# # save the output as a Numpy array
# np.save(open('C:/Users/hp/Desktop/Diabetic_retinopathy_dataset_kaggle/models/vgg16/bottle_neck_features/bottleneck_features_train_without_aug.npy', 'wb'), bottleneck_features_train)

In [6]:
# model_check_point_loc = 'C:/Users/hp/Desktop/Diabetic_retinopathy_dataset_kaggle/models/vgg16/vgg16_deep_feature_kg_without_SVD_dr.h5'
# #model_checkpoint = ModelCheckpoint(model_check_point_loc, monitor='val_acc', verbose=0, save_best_only=True, mode='max')

In [7]:
train_data = np.load(open('C:/Users/hp/Desktop/Diabetic_retinopathy_dataset_kaggle/models/vgg16/bottle_neck_features/bottleneck_features_train_without_aug.npy', 'rb'))
#test_data = np.load(open('D:/retinal_data_set_visioncare/models/vgg16/bottle_neck_features/bottleneck_features_test_without_aug.npy', 'rb'))

In [8]:
train_data_labels = train_generator.classes

In [9]:
# Feature Scaling - fature normalizing
scaler = StandardScaler()
X_train = scaler.fit_transform(train_data)

In [10]:
len(X_train)

744

In [11]:
train_data_labels = train_generator.classes  

X = X_train
Y = train_data_labels

In [12]:
len(Y)

744

In [13]:
np.unique(train_data_labels)

array([0, 1, 2, 3, 4])

In [14]:
class_weight = class_weight.compute_class_weight('balanced'
                                               ,np.unique(train_data_labels)
                                               ,train_data_labels)

In [15]:
dict(enumerate(class_weight))

{0: 0.3241830065359477,
 1: 1.488,
 2: 1.984,
 3: 3.381818181818182,
 4: 2.2545454545454544}

In [16]:
class_weight_dic = dict(enumerate(class_weight))

In [17]:
# define 5-fold cross validation test harness
#kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
kfold = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=25)
cvscores = []
trainScores = []
f1Score = []
num_k_folds = 5
fold_counter = 0
val_conmats = []
val_precisions = []
val_recalls = []
val_f_scores = []
input_dim = X_train.shape[1:][0]

rf_classifier = RandomForestClassifier(n_estimators=500,max_features=20,max_depth=5,min_samples_leaf=40,criterion="gini", class_weight=class_weight_dic, n_jobs=-1)

In [18]:
for train, test in kfold.split(X, Y):
    # Fit the model
    rf_classifier.fit(X[train], Y[train])
    
    y_train_pred = rf_classifier.predict(X[train])
    #y_train_pred = np.argmax(y_train_pred, axis=1)
    
    y_validation_pred = rf_classifier.predict(X[test])
    #y_validation_pred = np.argmax(y_validation_pred, axis=1)
    #y_validation_pred = np.argmax(y_validation_pred, axis=1)
    
    [precision, recall, f_score, _] = precision_recall_fscore_support(Y[test], y_validation_pred)
    #print("Validation k-fold #%d - precision: %f, recallL: %f, f-score: %f" % (fold_counter, precision, recall, f_score))
    
    conmat = confusion_matrix(Y[test], y_validation_pred)
    
    val_precisions.append(precision)
    val_recalls.append(recall)
    val_f_scores.append(f_score)
    val_conmats.append(conmat)
    fold_counter = fold_counter + 1
    
    trainScores.append(accuracy_score(Y[train], y_train_pred))
    cvscores.append(accuracy_score(Y[test], y_validation_pred))
    
print("\nAveraging the 5-fold results:")
print("%s: %.2f%%" % ('AVG Train Acc ', np.mean(trainScores) * 100))
print("%s: %.2f%%" % ('AVG Validation Acc ', np.mean(cvscores) * 100))
print("Validation precision - mean: %f, stddev: %f" % (np.mean(val_precisions), np.std(val_precisions)))
print("Validation recall - mean: %f, stddev: %f" % (np.mean(val_recalls), np.std(val_recalls)))
print("Validation f-score - mean: %f, stddev: %f" % (np.mean(val_f_scores), np.std(val_f_scores)))
print("Confusion matrix:")
print (sum(val_conmats).astype(float) / fold_counter)


Averaging the 5-fold results:
AVG Train Acc : 81.20%
AVG Validation Acc : 69.11%
Validation precision - mean: 0.639961, stddev: 0.217146
Validation recall - mean: 0.619823, stddev: 0.199361
Validation f-score - mean: 0.613703, stddev: 0.191226
Confusion matrix:
[[70.92 18.6   2.2   0.08  0.  ]
 [ 9.68  9.12  0.84  0.12  0.24]
 [ 3.88  2.2   6.44  1.76  0.72]
 [ 0.08  0.    0.44  5.24  3.04]
 [ 0.2   0.4   0.24  1.24 11.12]]
